In [1]:
import tensorflow as tf

# MNIST 데이터를 다운로드 합니다.
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("data/", one_hot=False)

W0806 14:43:17.757430 140664145954560 deprecation.py:323] From <ipython-input-1-a59898dfbfea>:5: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0806 14:43:17.758072 140664145954560 deprecation.py:323] From /home/teoac/solarisTutorial/lib/python3.5/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0806 14:43:17.758832 140664145954560 deprecation.py:323] From /home/teoac/solarisTutorial/lib/python3.5/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:262: extract_images (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be remove

Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz


W0806 14:43:17.984297 140664145954560 deprecation.py:323] From /home/teoac/solarisTutorial/lib/python3.5/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


Extracting data/t10k-labels-idx1-ubyte.gz


In [2]:
# 입력값과 출력값을 받기 위한 플레이스홀데를 정의합니다.
x = tf.placeholder(tf.float32, shape=[None, 784])
y = tf.placeholder(tf.int64, shape=[None])

In [3]:
# 변수들을 설정하고 소프트맥스 회귀 모델을 정의합니다.
W = tf.Variable(tf.zeros(shape=[784,10]), name='W')
b = tf.Variable(tf.zeros(shape=[10]), name='b')
logits = tf.matmul(x, W) + b
y_pred = tf.nn.softmax(logits)

In [4]:
# cross-engropy 손실 함수와 옵티마이저를 정의합니다.
#loss_function = -tf.reduce_sum(y*tf.log(y_pred), reduction_indices=[1])
loss_function = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
loss = tf.reduce_mean(loss_function)
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

# 텐서보드를 위한 요약 정보(scalar)를 정의합니다.
tf.summary.scalar("Loss", loss)

<tf.Tensor 'Loss:0' shape=() dtype=string>

In [5]:
# 세션을 열고 변수들에 초기값을 할당합니다.
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# 텐서보드 요약정보들을 하나로 합칩니다.
merged = tf.summary.merge_all()
# 텐서보드 summary 정보들을 저장할 폴더 경로를 설정합니다.
tensorboard_writer = tf.summary.FileWriter('./tensorboard_log', sess.graph)

In [6]:
for i in range(10000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x: batch_xs, y:batch_ys})
    summary = sess.run(merged, feed_dict={x: batch_xs, y: batch_ys})
    tensorboard_writer.add_summary(summary, i)
    if i%1000==999:
        print("#",i+1,": ", sess.run(loss_function, feed_dict={x:batch_xs, y:batch_ys}))

# 1000 :  [1.39431819e-01 8.94172769e-03 1.71573654e-01 1.74030414e-04
 3.07870185e-04 6.89005974e-05 6.50301483e-03 1.01701140e-01
 3.27193029e-02 4.35880989e-01 3.64594817e-01 9.41709077e-05
 6.42337441e-01 1.70931511e-04 2.92414546e-01 1.17564220e-02
 3.95100738e-04 1.11196591e-02 3.74401271e-01 6.70834333e-02
 4.54322580e-04 5.96762188e-02 1.85672119e-02 4.76615354e-02
 4.18533683e-01 2.53679585e-02 8.27685520e-02 1.96391165e-01
 2.45316140e-02 1.14509957e-02 1.10529352e-03 7.72973895e-03
 8.82699899e-03 2.80117933e-02 3.83184437e-04 1.15405591e-02
 3.45647190e-04 1.77127197e-01 6.77467138e-03 4.97638695e-02
 1.63366180e-02 4.68605869e-02 4.18510512e-02 1.50931925e-01
 1.10870808e-01 2.22725645e-02 1.44961551e-01 7.56343186e-01
 7.08851218e-03 3.83628048e-02 1.12344781e-02 2.39067785e-02
 1.21826798e-01 2.13133246e-01 1.87498634e-04 5.73160760e-02
 3.60459518e+00 1.92179624e-02 7.20387772e-02 6.97447479e-01
 1.77178683e-03 5.28043993e-02 1.66283135e-04 8.19144212e-03
 1.86233781e-0

In [7]:
# 학습이 끝나면 학습된 모델의 정확도를 출력합니다.
correct_prediction = tf.equal(tf.argmax(y_pred,1), y)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print('정확도: %f' %sess.run(accuracy, feed_dict={x:mnist.test.images, y:mnist.test.labels}))

정확도: 0.925400


In [18]:
print('W=', sess.run(W))
print('b=', sess.run(b))

W= [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
b= [-0.99454933  0.8931715   0.18379518 -0.5246859   0.02702699  2.2974827
 -0.38493288  1.3469313  -2.3486476  -0.49559042]


In [ ]:
sess.close()